In [29]:
import os
import zipfile
import glob
import pandas as pd
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
from keras.applications import VGG16, MobileNet, ResNet50
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Activation
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

# Dataset loading

In [3]:
# Set up Kaggle API Key
os.environ['KAGGLE_USERNAME'] = "XXXXXX"
os.environ['KAGGLE_KEY'] = "XXXXXX"

# Download the dataset
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification

# Unzip the dataset
with zipfile.ZipFile("muffin-vs-chihuahua-image-classification.zip", "r") as zip_ref:
    zip_ref.extractall(".")

muffin-vs-chihuahua-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


# Data preprocessing

### Reshape the images and convert to RGB

In [4]:
def load_images(image_dir, img_size):
    images = []
    for image_path in os.listdir(image_dir):
        img = Image.open(os.path.join(image_dir, image_path)).convert('RGB')  # Convert to RGB
        img_resized = img.resize((img_size, img_size))
        images.append(img_resized)
    return images


train_dir = 'train'
test_dir = 'test'
img_size = 224

# Load images
train_muffin_images = load_images(os.path.join(train_dir, 'muffin'), img_size)
train_chihuahua_images = load_images(os.path.join(train_dir, 'chihuahua'), img_size)
test_muffin_images = load_images(os.path.join(test_dir, 'muffin'), img_size)
test_chihuahua_images = load_images(os.path.join(test_dir, 'chihuahua'), img_size)


# Create DataFrames for train and test sets
train_data = {'image': train_muffin_images + train_chihuahua_images, 'label': ['muffin'] * len(train_muffin_images) + ['chihuahua'] * len(train_chihuahua_images)}
train_df = pd.DataFrame(train_data)

test_data = {'image': test_muffin_images + test_chihuahua_images, 'label': ['muffin'] * len(test_muffin_images) + ['chihuahua'] * len(test_chihuahua_images)}
test_df = pd.DataFrame(test_data)

# Print the DataFrames
print(train_df)
print(test_df)

                                                  image      label
0     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
1     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
2     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
3     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
4     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
...                                                 ...        ...
4728  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua
4729  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua
4730  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua
4731  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua
4732  <PIL.Image.Image image mode=RGB size=224x224 a...  chihuahua

[4733 rows x 2 columns]
                                                  image      label
0     <PIL.Image.Image image mode=RGB size=224x224 a...     muffin
1     <PIL.Image.Image image mode=RGB

### Convert the images into np arrays

In [5]:
def images_to_arrays(images):
    return np.stack([np.array(img) for img in images])

# vertical concatenatenation of arrays along the first axis
X_train = np.vstack((images_to_arrays(train_muffin_images), images_to_arrays(train_chihuahua_images)))
X_test = np.vstack((images_to_arrays(test_muffin_images), images_to_arrays(test_chihuahua_images)))

y_train = np.array([0] * len(train_muffin_images) + [1] * len(train_chihuahua_images))
y_test = np.array([0] * len(test_muffin_images) + [1] * len(test_chihuahua_images))

# Normalize the pixel values
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Network architectures

## Training the model with 5-Fold CV

During the model.fit() both loss and accuracy for the training and for the validation set are computed, in order to monitoring the possible cases of overfitting. If the loss value decreases while the val_loss value starts to increase, it may indicate that the model is overfitting to the training set and is not generalizing well to new data.

### General model creation and training

In [6]:
def create_model(base_model):
    for layer in base_model.layers: #transfer learning
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x) #reduces the spatial dimensions of the output
    x = Dense(1024)(x) 
    x = BatchNormalization()(x) #tries to make the process faster
    x = Activation('relu')(x) #introduces non-linearity
    predictions = Dense(1, activation='sigmoid')(x) #produces the binary classification output
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [7]:
def train_and_evaluate_model(create_model, learning_rate, epochs, batch_size) : #these hyperparameters will be changed for experimenting
    # KFold cross-validation
    num_folds = 5
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    fold_scores = []
    fold_accuracies = []

    for train_index, val_index in kfold.split(X_train):
        X_train_fold, X_val = X_train[train_index], X_train[val_index]
        y_train_fold, y_val = y_train[train_index], y_train[val_index]

        # Create and compile the model
        model = create_model
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

        # Train the model on the current fold
        history = model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

        # Evaluate the model on the test set and store the performance metric (using zero-one loss as required)
        y_pred = model.predict(X_test)
        y_pred = np.round(y_pred)
        zero_one_loss = 1 - np.mean(y_pred.flatten() == y_test)
        fold_scores.append(zero_one_loss)
        fold_accuracies.append(np.mean(history.history['val_accuracy']))

    average_zero_one_loss = np.mean(fold_scores)
    average_accuracy = np.mean(fold_accuracies)
    print(f"Average zero-one loss across {num_folds} folds: {average_zero_one_loss}")
    print(f"Average accuracy across {num_folds} folds: {average_accuracy}")

    return average_zero_one_loss, average_accuracy

### VGG16

In [63]:
base_vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

In [65]:
vgg_loss_1, vgg_accuracy_1 = train_and_evaluate_model(create_model(base_vgg_model), learning_rate=0.001, epochs=4, batch_size=8)

Epoch 1/4
474/474 [==============================] - 386s 813ms/step - loss: 0.2101 - accuracy: 0.9218 - val_loss: 0.1063 - val_accuracy: 0.9620
Epoch 2/4
474/474 [==============================] - 377s 796ms/step - loss: 0.1422 - accuracy: 0.9480 - val_loss: 0.0547 - val_accuracy: 0.9820
Epoch 3/4
474/474 [==============================] - 372s 785ms/step - loss: 0.1087 - accuracy: 0.9609 - val_loss: 1.0950 - val_accuracy: 0.6822
Epoch 4/4
37/37 [==============================] - 91s 2s/step
Epoch 1/4
474/474 [==============================] - 388s 804ms/step - loss: 0.1252 - accuracy: 0.9517 - val_loss: 0.0492 - val_accuracy: 0.9831
Epoch 2/4
474/474 [==============================] - 535s 1s/step - loss: 0.1062 - accuracy: 0.9614 - val_loss: 0.0728 - val_accuracy: 0.9704
Epoch 3/4
474/474 [==============================] - 477s 1s/step - loss: 0.0919 - accuracy: 0.9654 - val_loss: 0.0385 - val_accuracy: 0.9884
Epoch 4/4
37/37 [==============================] - 92s 2s/step
Epoch 1/4


In [66]:
vgg_loss_2, vgg_accuracy_2 = train_and_evaluate_model(create_model(base_vgg_model), learning_rate=0.0001, epochs=8, batch_size=16)

Epoch 1/8
237/237 [==============================] - 378s 2s/step - loss: 0.1752 - accuracy: 0.9250 - val_loss: 0.2093 - val_accuracy: 0.9694
Epoch 2/8
237/237 [==============================] - 373s 2s/step - loss: 0.0995 - accuracy: 0.9638 - val_loss: 0.0733 - val_accuracy: 0.9831
Epoch 3/8
237/237 [==============================] - 373s 2s/step - loss: 0.0808 - accuracy: 0.9731 - val_loss: 0.0511 - val_accuracy: 0.9863
Epoch 4/8
237/237 [==============================] - 377s 2s/step - loss: 0.0652 - accuracy: 0.9781 - val_loss: 0.0556 - val_accuracy: 0.9831
Epoch 5/8
237/237 [==============================] - 443s 2s/step - loss: 0.0575 - accuracy: 0.9834 - val_loss: 0.0558 - val_accuracy: 0.9820
Epoch 6/8
237/237 [==============================] - 389s 2s/step - loss: 0.0569 - accuracy: 0.9786 - val_loss: 0.0430 - val_accuracy: 0.9873
Epoch 7/8
237/237 [==============================] - 391s 2s/step - loss: 0.0438 - accuracy: 0.9865 - val_loss: 0.0504 - val_accuracy: 0.9810
Epoch 

In [11]:
best_vgg_accuracy = max(vgg_accuracy_1, vgg_accuracy_2)
best_vgg_loss = min(vgg_loss_1, vgg_loss_2)

### MobileNet

In [9]:
base_mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

In [69]:
mobilenet_loss_1, mobilenet_accuracy_1 = train_and_evaluate_model(create_model(base_mobilenet_model), learning_rate=0.001, epochs=4, batch_size=8)

Epoch 1/4
474/474 [==============================] - 38s 77ms/step - loss: 0.1193 - accuracy: 0.9643 - val_loss: 0.0390 - val_accuracy: 0.9905
Epoch 2/4
474/474 [==============================] - 37s 79ms/step - loss: 0.0676 - accuracy: 0.9775 - val_loss: 0.0280 - val_accuracy: 0.9905
Epoch 3/4
474/474 [==============================] - 36s 76ms/step - loss: 0.0643 - accuracy: 0.9789 - val_loss: 0.0304 - val_accuracy: 0.9884
Epoch 4/4
37/37 [==============================] - 7s 189ms/step
Epoch 1/4
474/474 [==============================] - 41s 84ms/step - loss: 0.0494 - accuracy: 0.9844 - val_loss: 0.0068 - val_accuracy: 0.9989
Epoch 2/4
474/474 [==============================] - 37s 78ms/step - loss: 0.0524 - accuracy: 0.9855 - val_loss: 0.0071 - val_accuracy: 0.9979
Epoch 3/4
474/474 [==============================] - 36s 76ms/step - loss: 0.0562 - accuracy: 0.9820 - val_loss: 0.0125 - val_accuracy: 0.9968
Epoch 4/4
37/37 [==============================] - 8s 214ms/step
Epoch 1/4
47

In [10]:
mobilenet_loss_2, mobilenet_accuracy_2 = train_and_evaluate_model(create_model(base_mobilenet_model), learning_rate=0.0001, epochs=8, batch_size=16)

Epoch 1/8
237/237 [==============================] - 33s 133ms/step - loss: 0.0751 - accuracy: 0.9749 - val_loss: 0.0303 - val_accuracy: 0.9894
Epoch 2/8
237/237 [==============================] - 31s 132ms/step - loss: 0.0294 - accuracy: 0.9902 - val_loss: 0.0210 - val_accuracy: 0.9916
Epoch 3/8
237/237 [==============================] - 31s 133ms/step - loss: 0.0179 - accuracy: 0.9947 - val_loss: 0.0200 - val_accuracy: 0.9926
Epoch 4/8
237/237 [==============================] - 32s 133ms/step - loss: 0.0153 - accuracy: 0.9955 - val_loss: 0.0243 - val_accuracy: 0.9937
Epoch 5/8
237/237 [==============================] - 31s 133ms/step - loss: 0.0128 - accuracy: 0.9971 - val_loss: 0.0194 - val_accuracy: 0.9937
Epoch 6/8
237/237 [==============================] - 32s 136ms/step - loss: 0.0101 - accuracy: 0.9971 - val_loss: 0.0185 - val_accuracy: 0.9916
Epoch 7/8
237/237 [==============================] - 32s 135ms/step - loss: 0.0111 - accuracy: 0.9968 - val_loss: 0.0238 - val_accuracy:

In [12]:
best_mobilenet_accuracy = max(mobilenet_accuracy_1, mobilenet_accuracy_2)
best_mobilenet_loss = min(mobilenet_loss_1, mobilenet_loss_2)

### ResNet50

In [13]:
base_resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

In [14]:
resnet_loss_1, resnet_accuracy_1 = train_and_evaluate_model(create_model(base_resnet_model), learning_rate=0.001, epochs=4, batch_size=8)

Epoch 1/4
474/474 [==============================] - 169s 351ms/step - loss: 0.5140 - accuracy: 0.7768 - val_loss: 3.9518 - val_accuracy: 0.4636
Epoch 2/4
474/474 [==============================] - 164s 345ms/step - loss: 0.4428 - accuracy: 0.8043 - val_loss: 1.4141 - val_accuracy: 0.5639
Epoch 3/4
474/474 [==============================] - 171s 361ms/step - loss: 0.4159 - accuracy: 0.8154 - val_loss: 0.5577 - val_accuracy: 0.8015
Epoch 4/4
37/37 [==============================] - 36s 960ms/step
Epoch 1/4
474/474 [==============================] - 165s 343ms/step - loss: 0.4097 - accuracy: 0.8222 - val_loss: 2.2881 - val_accuracy: 0.4857
Epoch 2/4
474/474 [==============================] - 159s 336ms/step - loss: 0.3894 - accuracy: 0.8230 - val_loss: 0.3807 - val_accuracy: 0.8585
Epoch 3/4
474/474 [==============================] - 162s 341ms/step - loss: 0.3838 - accuracy: 0.8347 - val_loss: 0.5755 - val_accuracy: 0.6843
Epoch 4/4
37/37 [==============================] - 34s 905ms/ste

In [15]:
resnet_loss_2, resnet_accuracy_2 = train_and_evaluate_model(create_model(base_resnet_model), learning_rate=0.0001, epochs=8, batch_size=16)

Epoch 1/8
237/237 [==============================] - 163s 675ms/step - loss: 0.4460 - accuracy: 0.7987 - val_loss: 0.6333 - val_accuracy: 0.5533
Epoch 2/8
237/237 [==============================] - 155s 653ms/step - loss: 0.3594 - accuracy: 0.8447 - val_loss: 0.5017 - val_accuracy: 0.7666
Epoch 3/8
237/237 [==============================] - 150s 633ms/step - loss: 0.3280 - accuracy: 0.8568 - val_loss: 0.7815 - val_accuracy: 0.6410
Epoch 4/8
237/237 [==============================] - 154s 649ms/step - loss: 0.3036 - accuracy: 0.8719 - val_loss: 0.3985 - val_accuracy: 0.8237
Epoch 5/8
237/237 [==============================] - 153s 647ms/step - loss: 0.2880 - accuracy: 0.8825 - val_loss: 0.5200 - val_accuracy: 0.7835
Epoch 6/8
237/237 [==============================] - 153s 647ms/step - loss: 0.2728 - accuracy: 0.8872 - val_loss: 2.0171 - val_accuracy: 0.4815
Epoch 7/8
237/237 [==============================] - 156s 657ms/step - loss: 0.2632 - accuracy: 0.8941 - val_loss: 0.6548 - val_ac

In [16]:
best_resnet_accuracy = max(resnet_accuracy_1, resnet_accuracy_2)
best_resnet_loss = min(resnet_loss_1, resnet_loss_2)

## Results

In [35]:
print(f"Best VGG accuracy: {best_vgg_accuracy}\nBest MobileNet accuracy: {best_mobilenet_accuracy}\nBest ResNet accuracy: {best_resnet_accuracy}")

Best VGG accuracy: 0.9950369313359261
Best MobileNet accuracy: 0.9983896523714065
Best ResNet accuracy: 0.7815316691994667


In [36]:
print(f"Best VGG loss: {best_vgg_loss}\nBest MobileNet loss: {best_mobilenet_loss}\nBest ResNet loss: {best_resnet_loss}")

Best VGG loss: 0.0153716216216216
Best MobileNet loss: 0.002195945945945921
Best ResNet loss: 0.1609797297297297


In [27]:
best_accuracy = max(best_vgg_accuracy, best_mobilenet_accuracy, best_resnet_accuracy)
best_loss = min(best_vgg_loss, best_mobilenet_loss, best_resnet_loss)

# Define the architecture names
architecture_names = ['VGG', 'MobileNet', 'ResNet']

# Initialize variables to store the best accuracy and loss
best_accuracy = 0.0
best_loss = float('inf')
best_accuracy_architecture = ''
best_loss_architecture = ''

# Iterate over the architectures
for i, architecture in enumerate([best_vgg_accuracy, best_mobilenet_accuracy, best_resnet_accuracy]):
    # Check if the current architecture has the best accuracy
    if architecture > best_accuracy:
        best_accuracy = architecture
        best_accuracy_architecture = architecture_names[i]
        
for i, architecture_loss in enumerate([best_vgg_loss, best_mobilenet_loss, best_resnet_loss]):
    # Check if the current architecture has the best loss
    if architecture_loss < best_loss:
        best_loss = architecture_loss
        best_loss_architecture = architecture_names[i]

# Print the architecture with the best accuracy and best loss
print(f"Best accuracy: {best_accuracy} from {best_accuracy_architecture}")
print(f"Best loss: {best_loss} from {best_loss_architecture}")

Best accuracy: 0.9983896523714065 from MobileNet
Best loss: 0.002195945945945921 from MobileNet
